## Processing video data for VGG model

VGG needs 224x224 videos so its data is processed separately

In [ ]:
import dlib
import numpy as np
import skvideo.io
import os
from video_processes import *
import pickle

In [ ]:
p = "shape_predictor_68_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor(p)

<h3> Focus Videos </h3>

In [ ]:
####
#parameters
folder_list = ["Focus", "Distract"] #list of video folders. folder name will be used as labels
resample_center_step = 12 #step to skip frames for facial center and max distances to boundary
resample_crop_step = 2 #step to skip frames for storing after cropping
cropped_size = 224 #size of cropped square video

####
#functional codes
pr_videos = []

for folder in folder_list:
    #get video list
    allfiles = os.listdir(folder+'/')
    #list to store processed videos
    for fn in allfiles:
        try: #try open videos
            v = skvideo.io.vread(folder + '/' + fn)
            H, W = v.shape[1:-1]   #video dimension
        except:
            continue
        #roughly query facial center and max distance to boundaries in videos
        centers = []
        mds = []
        processed_video = []
        for f in v[::resample_center_step]:
            try:
                c, md = get_face_region(f, detector, predictor, 20)
                centers.append(c)
                mds.append(md)
            except:
                continue
        #using median center and max of max distances
        C = np.int32(np.median(np.array(centers),axis=0))
        M = np.int32(np.max(mds))
        #resample and crop to square videos
        for frame in v[::resample_crop_step]:
            cropped_frame = squarize_frame(frame, C, M, (cropped_size,cropped_size))
            processed_video.append(cropped_frame)
        processed_video = np.array(processed_video, dtype=np.uint8)
        #store processed video
        pr_videos.append(processed_video)
        #store label
        labels.append(folder)

### Save data

In [ ]:
of = open('processed_videos_224.obj', 'wb')
pickle.dump([read_files, pr_videos], of)
of.close()